# Laboratório 4

* *Daniel Guimarães*

* *Iury Saboia*

# 1 : Protocolo ideal

## 1.1
**(Alice)** : Crie um circuito quântico de dois qubits (q0 e q1), e por enquanto ignore q1.
Implemente uma operação unitária *Ua* que seja capaz de receber como entrada dois bits clássicos.
Para cada combinação possível desses 2 bits, um dos estados { |0⟩, |+⟩, |1⟩, |-⟩ } deve ser obtido
após a aplicação de *Ua* em q0.

In [190]:
from qiskit import QuantumCircuit

def ua(qc: QuantumCircuit, c0: int, c1: int, duplo=False):
    """
    Cria um estado de acordo com as entradas classicas c0 e c1

    c0 = 0, c1 = 0:
        retorna |0⟩
    c0 = 0, c1 = 1:
        retorna |1⟩
    c0 = 1, c1 = 0:
        retorna |+⟩
    c0 = 1, c1 = 1:
        retorna |-⟩
    """
    if not c0 and not c1:
        pass
    elif not c0 and c1:
        qc.x(0)
        if duplo: qc.x(1)
    elif c0 and not c1:
        qc.h(0)
        if duplo: qc.h(1)
    else:
        qc.x(0)
        qc.h(0)
        if duplo:
            qc.x(1)
            qc.h(1)

## 1.2
**(Bob)** : Crie uma operação unitária *Ub* que receba como parâmetro um bit clássico e
realize uma medida em *q0*. Dependendo do bit recebido, a medida deve corresponder a uma
medida projetiva na base computacional ou na base diagonal.

In [191]:
from qiskit import Aer, execute

simulator = Aer.get_backend('qasm_simulator')

def ub(qc: QuantumCircuit, c: int, qubit=0) -> int:
    if c:
        qc.h(qubit)
    qc.measure(qubit, 0)
    resultado = execute(qc, simulator, shots=1).result()
    return int(next(iter(resultado.to_dict()['results'][0]['data']['counts']))[-1])

## 1.3
**(Conciliação de bases)** : Implemente um algoritmo (clássico) de conciliação de bases,
de modo que após a aplicação do algoritmo Alice e Bob permanecem com bits relacionados
a escolhas de mesma base


In [192]:
from typing import Tuple, List

# lista = [qc1, qc2, qc3]
# alice = [00 (0) , 10  (+) , 11  (-) ]
# bob   = [0  (01), 0   (01), 1   (+-)]

def conciliacao_bases(lista_bob: List[int], escolhas_alice: List[Tuple[int, int]], escolha_bob: List[int]) -> Tuple:
    """Retorna as tres novas listas"""

    lista_final = []
    escolhas_alice_final = []
    escolhas_bob_final = []

    for i in range(len(lista_bob)):                 # para cada item das listas
        if escolhas_alice[i][0] == escolha_bob[i]:  # se bob e alice escolheram iguais
            lista_final.append(lista_bob[i])                    # salva nas listas finais
            escolhas_alice_final.append(escolhas_alice[i])
            escolhas_bob_final.append(escolha_bob[i])

    # atualiza as listas
    return lista_final, escolhas_alice_final, escolhas_bob_final

## 1.4
**(Verificacao da taxa de erro)** : Implemente um novo algoritmo que sacrifica 10% dos bits de Alice e Bob e
compara seus valores, de modo a estimar a taxa de erro

In [193]:
def taxa_de_erro(lista_final: List[int], escolhas_alice: List[Tuple[int, int]]):

    # obsevado_bob =     [0 , 1 , 0 , 0 , 1 ]

    # escolhas_alice =  [00, 01, 10, 11, 01]
    #                   [0 , 1 , + , - , 1 ]
    soma = 0
    total = len(lista_final) // 10
    for i in range(total):
        alice = escolhas_alice.pop()
        final = lista_final.pop()
        if alice[1] != final:
            soma += 1

    return soma / total

**(Primeira rodada do protocolo BB84)** : Rode o protocolo para N = 10000 qubits enviados por Alice,
onde as escolhas de bits (tanto por Alice quanto por Bob) são aleatórias.
Qual a taxa de erro encontrada?

In [194]:
from random import randint

def protocolo_bb84(n=1000):
    observado_bob: List[int] = []
    escolhas_alice: List[Tuple[int, int]] = []
    escolhas_bob: List[int] = []

    for _ in range(n):
        # bits aleatorios
        c0_alice, c1_alice = randint(0, 1), randint(0, 1)
        c_bob = randint(0, 1)

        # operacoes
        qc = QuantumCircuit(2, 2)
        ua(qc, c0_alice, c1_alice)
        x = ub(qc, c_bob)

        # salvando operacoes
        observado_bob.append(x)
        escolhas_alice.append((c0_alice, c1_alice))
        escolhas_bob.append(c_bob)

    observado_bob, escolhas_alice, escolhas_bob = conciliacao_bases(observado_bob, escolhas_alice, escolhas_bob)

    erro = taxa_de_erro(observado_bob, escolhas_alice)
    print(f"taxa de erro: {(erro * 100):.2f}%")

protocolo_bb84()

taxa de erro: 0.00%


# Parte 2: Ataque Intercept-Resend

## 2.1
**(Eva)** : Implemente agora a ação da espiã, Eva, no protocolo acima. Para cada qubit
enviado por Alice, Eva deve "interceptar" o qubit e proceder exatamente como Bob; em
seguida ela deve reenviar o resultado de sua medida projetiva para Bob. Utilize agora
o qubit q1 para representar o qubit reenviado por Eva a Bob, e modifique o circuito
anterior de modo que agora Bob mede q1 ao invés de q0.

In [195]:
def ua_duplicado(qc: QuantumCircuit, c0: int, c1: int):
    return ua(qc, c0, c1, True)

def ub_modificado(qc: QuantumCircuit, c: int):
    return ub(qc, c, 1)

def ue(qc: QuantumCircuit) -> int:
    from random import choice
    base_escolhida = choice([0, 1])
    if base_escolhida == 1:
        qc.h(1)
        qc.measure(1, 0)
        qc.h(1)
    else:
        qc.measure(1, 0)
    return base_escolhida

## 2.2
**(Segunda rodada do protocolo BB84)** : Rode o protocolo para n = 10000 qubits enviados por Alice.
Qual a nova taxa de erro encontrada no final?

In [196]:
def protocolo_bb84_modificado(n=10000):
    observado_bob: List[int] = []
    escolhas_alice: List[Tuple[int, int]] = []
    escolhas_bob: List[int] = []

    for _ in range(n):
        # bits aleatorios
        c0_alice, c1_alice = randint(0, 1), randint(0, 1)
        c_bob = randint(0, 1)

        # operacoes
        qc = QuantumCircuit(2, 1)
        ua_duplicado(qc, c0_alice, c1_alice)
        ue(qc)
        x = ub_modificado(qc, c_bob)

        # salvando operacoes
        observado_bob.append(x)
        escolhas_alice.append((c0_alice, c1_alice))
        escolhas_bob.append(c_bob)

    observado_bob, escolhas_alice, escolhas_bob = conciliacao_bases(observado_bob, escolhas_alice, escolhas_bob)

    erro = taxa_de_erro(observado_bob, escolhas_alice)
    print(f"taxa de erro: {(erro * 100):.2f}%")

protocolo_bb84_modificado()

taxa de erro: 24.30%


## 2.3
**(Taxa de erro máxima)** : Eva sabe que o protocolo é interrompido se a taxa de erro ultrapassar o limite de,
digamos, 10%. Qual a maior fração de qubits de Alice que ela deve interceptar de modo a sempre manter a taxa de
erro abaixo do limite? E qual a fração dos bits da chave compartilhada que Eva conhece? Simule o caso.

In [197]:
# 10 / 25 = 40% dos bits que ela pode pegar

def conciliacao_bases_eva(lista_bob: List[int], escolhas_alice: List[Tuple[int, int]], escolha_bob: List[int], escolhas_eva) -> Tuple:
    """Retorna as tres novas listas"""

    lista_final = []
    escolhas_alice_final = []
    escolhas_bob_final = []
    escolhas_eva_final = []

    for i in range(len(lista_bob)):                 # para cada item das listas
        if escolhas_alice[i][0] == escolha_bob[i]:  # se bob e alice escolheram iguais
            lista_final.append(lista_bob[i])                    # salva nas listas finais
            escolhas_alice_final.append(escolhas_alice[i])
            escolhas_bob_final.append(escolha_bob[i])
            escolhas_eva_final.append(escolhas_eva[i])

    # atualiza as listas
    return lista_final, escolhas_alice_final, escolhas_bob_final, escolhas_eva_final

def protocolo_bb84_eva_limite(n=10000):
    from random import uniform
    observado_bob: List[int] = []
    escolhas_alice: List[Tuple[int, int]] = []
    escolhas_bob: List[int] = []
    escolhas_eva: List[int] = []

    for _ in range(n):
        # bits aleatorios
        c0_alice, c1_alice = randint(0, 1), randint(0, 1)
        c_bob = randint(0, 1)

        # operacoes
        qc = QuantumCircuit(2, 1)
        ua_duplicado(qc, c0_alice, c1_alice)
        if uniform(0, 1) < 0.4:
            escolha_eva = ue(qc)
            escolhas_eva.append(escolha_eva)
        else:
            escolhas_eva.append(-1)
        x = ub_modificado(qc, c_bob)

        # salvando operacoes
        observado_bob.append(x)
        escolhas_alice.append((c0_alice, c1_alice))
        escolhas_bob.append(c_bob)

    observado_bob, escolhas_alice, escolhas_bob, escolhas_eva = conciliacao_bases_eva(observado_bob, escolhas_alice, escolhas_bob, escolhas_eva)

    soma_conhecidos_eva = 0
    for i in range(len(escolhas_eva)):
        if escolhas_eva[i] == -1:       # nao viu esse bit
            continue
        if escolhas_eva[i] == escolhas_alice[i][0]:     # pegou a base correta
            soma_conhecidos_eva += 1
    conhecidos_eva = soma_conhecidos_eva / len(escolhas_eva)
    erro = taxa_de_erro(observado_bob, escolhas_alice)
    print(f"taxa de erro: {(erro * 100):.2f}%\ntaxa de conhecimento da eva: {(conhecidos_eva * 100):.2f}%")

protocolo_bb84_eva_limite()

taxa de erro: 10.12%
taxa de conhecimento da eva: 20.43%


## 2.4
**(Amplificação de Privacidade)** Efetua uma etapa de um algoritmo de Amplificação de Privacidade,
que realiza uma operação soma módulo 2 (XOR) entre cada 2 bits consecutivos da chave compartilhada.
Com base nesse resultado, comente sobre a etapa de Amplificação de Privacidade e sua importância para o
protocolo BB84

In [199]:
# basicamente, apesar de que a chave possui somente a metade do seu comprimento original, é necessário que a Eva
# saiba dois bits consecutivos da chave, e as chances de a Eva acertar dois seguidos fica extremamente pequena, pois:
#   1. Se ela vai ver sempre dois bits seguidos, ela vai ver em geral menos da chave, pois ela tem que continuar vendo
#      somente 40% das vezes. Ou seja, agora ela irá observar 20% das vezes (mas duas vezes seguidas)
#   2. A chance dela de acertar o bit em 50% das vezes agora foi para 25% das vezes.

# Sobre as vantagens, o ponto 1 mostra que a Eva irá intervir menos vezes, o que significa que a taxa de erro (no geral,
# considerando 2bits = 1 bit da chave) irá diminuir.
# Além disso, os bits da chave que ela consegue ver serão ainda menores, o que é importante para usos como o "one time pad"

def protocolo_bb84_eva_privacidade(n=10000):
    from random import uniform
    observado_bob: List[int] = []
    escolhas_alice: List[Tuple[int, int]] = []
    escolhas_bob: List[int] = []
    escolhas_eva: List[int] = []
    eva_olha_duas_vezes = False

    for _ in range(n):
        # bits aleatorios
        c0_alice, c1_alice = randint(0, 1), randint(0, 1)
        c_bob = randint(0, 1)

        # operacoes
        qc = QuantumCircuit(2, 1)
        ua_duplicado(qc, c0_alice, c1_alice)
        if uniform(0, 1) < (0.4 / 2) or eva_olha_duas_vezes:
            eva_olha_duas_vezes = not eva_olha_duas_vezes   # faz com que ela olhe duas vezes seguidas
            escolha_eva = ue(qc)
            escolhas_eva.append(escolha_eva)
        else:
            escolhas_eva.append(-1)
        x = ub_modificado(qc, c_bob)

        # salvando operacoes
        observado_bob.append(x)
        escolhas_alice.append((c0_alice, c1_alice))
        escolhas_bob.append(c_bob)

    observado_bob, escolhas_alice, escolhas_bob, escolhas_eva = conciliacao_bases_eva(observado_bob, escolhas_alice, escolhas_bob, escolhas_eva)

    soma_conhecidos_eva = 0
    for j in range(len(escolhas_eva) // 2):
        i, i2 = j * 2, j * 2 + 1

        if escolhas_eva[i] == -1 or escolhas_eva[i2] == -1:       # nao viu esse bit
            continue
        if escolhas_eva[i] == escolhas_alice[i][0] and escolhas_eva[i2] == escolhas_alice[i2][0]:     # pegou a base correta
            soma_conhecidos_eva += 1

    conhecidos_eva = soma_conhecidos_eva / (len(escolhas_eva) // 2)
    erro = taxa_de_erro(observado_bob, escolhas_alice)
    print(f"taxa de erro: {(erro * 100):.2f}%\ntaxa de conhecimento da eva: {(conhecidos_eva * 100):.2f}%")

protocolo_bb84_eva_privacidade()

taxa de erro: 9.86%
taxa de conhecimento da eva: 3.58%
